# AGUF for a set of maps of scans.

Runs AGUF fitting on a map of scans
see Jensen, A. B., Christensen, T. E. K., Weninger, C. & Birkedal, H. (2022). J. Synchrotron Rad. 29, 1420-1428. (https://doi.org/10.1107/S1600577522008232)

In [ ]:
#Import necessary libaries

%matplotlib widget
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy.ndimage as ndimage
import json
#To import DanMAX from the folder above:
sys.path.append('../')
import DanMAX as DM
    
style = DM.darkMode(style_dic={'figure.figsize':'large'})

def flatten(maps, nanangles, bkgtth, datatth,local_bkg=True):
    if local_bkg:
        flat_data = np.mean(maps['cake_map'][:,:,:,datatth],axis=3) -np.mean(maps['cake_map'][:,:,:,bkgtth],axis=3)
    else:
        flat_data = np.mean(maps['cake_map'][:,:,:,datatth],axis=3)
    flat_data[:,:,nanangles] = np.nan
    azi = maps['azi']*np.pi/180
    return flat_data, azi

## Define samples as groups of individuals of scans

In [ ]:

groups = DM.getProposalScans()
proposal,visit = DM.getCurrentProposal() 

# Single scan AGUF

Find the `xrd_range` parameter in the XRD_map notebook.
The parameter should be from a little before to a little after the peak.

It limits how much data will be loaded, but it needs to contain so local background, so it can be subtracted.

In [ ]:
scans = groups[list(groups.keys())[0]][list(groups[list(groups.keys())[0]].keys())[0]]
xrd_range = [0.8,1.2]
azi_range = None # chi values of interest
maps = DM.mapping.stitchScans(scans,XRF=False,XRD_cake=True,proposal=proposal,visit=visit,xrd_range=xrd_range,azi_range=azi_range)
maps['cake_map'] = (maps['cake_map'].transpose(2,3,0,1)/ maps['I0_map']).transpose(2,3,0,1)

### Find the angles without data. should be any angle containing a pure 0 from the mask, as it can mess up the averages.

Write a list of the indexes corrosponding to the bad angles. should be comma separated in a `np.r_[]`

The grap below will be zero or one. anywhere that is either 1 or touching a 1 should be excluded. 
Remember that python ranges don't include the last symbol.

so if indexes 5--10, and 55--78 [inclusive] are zeroes, nanangles should be set as follows:

`nanangles = np.r_[4:12,54:80]`

Otherwise you can use the printed guess

In [ ]:
plt.figure()
plt.plot(np.sum(maps['cake_map']==0,axis=(0,1,3))>0)

nanangles_guess = np.arange(0,maps['cake_map'].shape[2])[np.sum(maps['cake_map']==0,axis=(0,1,3))>0]
print(nanangles_guess)

In [ ]:
nanangles = np.r_[0:32,
                59:74,
                105:147,
                175:180]

### Find indexes for use as diffraction signal and for background

Use the graph below to find which indexes should be used for background and which to use as data

Remember again that the first index is included, and the last is excluded.

So to use indexes 0–4 and 15–19 [inclusive] as background, and 5–14 as data set in code:

`bkgtth = np.r_[0:5,15:20]`

`datatth = np.r_[5:15]`

In [ ]:
plt.figure()
plt.plot(np.sum(maps['cake_map'],axis=(0,1,2)))


In [ ]:
bkgtth = np.r_[0:4,10:14] # indexes from background
datatth = np.s_[4:10] #Indexes for data in loaded data.


### Find Phase background
Use the following two cell to set a good threshold.
It should be high enough that air/kapton is not included in the fit, as this can drastically slow down the process.

It should hovever not be so high that Important features are lost. Idealy the limit should also work across multiple samples.
use the parameter `phase_mask_limit`

In [ ]:
flat,chi = flatten(maps,nanangles,bkgtth,datatth,False)
phase_mask_limit = 5000
plt.figure()
plt.imshow(np.nansum(flat,axis=2)>phase_mask_limit)

## Run single scan AGUF.

Run AGUF and show the figure.

In [ ]:
flat,chi = flatten(maps,nanangles,bkgtth,datatth,False)
mask = np.nansum(flat,axis=2)>8000


fitmap = DM.fitting.fitMesh(DM.fitting.circleGaussFit,chi,flat,mask,verbose=True)
fit_params,names = DM.fitting.parseFitMesh(DM.fitting.parseCircleGaussFit,fitmap)

In [ ]:

Hue = fit_params[:,:,0]
DoO = fit_params[:,:,1]
I_tot = fit_params[:,:,2]

aguf_im = DM.mapping.combineMaps((Hue+0.5) % 1,DoO,I_tot/np.max(I_tot),True,[[10,15],[10,49]])
plt.figure()
plt.imshow(aguf_im)

# Bulk AGUF

First set the parameters for every peak.
The code above is used to find the settings for a single peak.
The following allows for bulk computations of multiple peaks and multiple samples.

You can use a mask based on the mask_maker notebook, or choose to set a threshold for every peak using the `phase_mask_limit`
To use the phase specific limit, set `use_phase_mask: True`.
The saved mask is best if you have colocalized phases, otherwise use the phase_mask.


Then run the next two cells to run AGUF on all peaks for all samples and groups in the groups variable

In [ ]:
peaks = {
    '002': {
        'xrd_range': [7.95,8.55], # tth/q values for the peak of interest
        'azi_range': None, # chi values of interest:
        'nanangles':np.r_[9:24,
                        71:83,
                        92:97,
                        98:106,
                        117:126,
                        144:153,
                        156:162,
                        173:177], # Angles not to use doe to lines on detector Find elsewhere
        'bkgtth': np.r_[0:6,14:20], # indexes from background
        'datatth': np.s_[6:14], #Indexes for data in loaded data.
        'local_background': True, # Use local background, true for diffraction
        'phase_mask_limit': 100, # Limit data must be above
        'chi_shift': 0, # shift applied to chi before plotting
        'use_phase_mask': True, # Create mask from loaded data and phase_mask_limit
    },
    'saxs': {
        'xrd_range': [0.8,1.2], # tth/q values for the peak of interest
        'azi_range': None, # chi values of interest:
        'nanangles':np.r_[0:32,
                            59:74,
                            105:147,
                            175:180], # Angles not to use doe to lines on detector Find elsewhere
        'bkgtth': np.r_[0:4,10:14], # indexes from background
        'datatth': np.s_[4:10], #Indexes for data in loaded data.
        'local_background': False, # Use local background, true for diffraction
        'phase_mask_limit': 8000, # Limit data must be above
        'chi_shift': 0.5, # shift applied to chi before plotting
        'use_phase_mask': True, # Create mask from loaded data and phase_mask_limit
    },
}


## Run AGUF

Runs AGUF on all datasets and saves the results in process/aguf_fits

In [ ]:
for peak in peaks:
    print(f'Fitting peak: {peak}')
    xrd_range = peaks[peak]['xrd_range']
    azi_range = peaks[peak]['azi_range']
    nanangles = peaks[peak]['nanangles']
    bkgtth = peaks[peak]['bkgtth']
    datatth = peaks[peak]['datatth']
    phase_mask_limit = peaks[peak]['phase_mask_limit']
    use_phase_mask = peaks[peak]['use_phase_mask']
    local_background = peaks[peak]['local_background']
    for group in groups:
        print(f'\tWorking on group: {group}')
        file_exists = False
        for animal in groups[group]:
            print(f'\t\tFitting animal: {animal}')
            scans = groups[group][animal]
            maps = DM.mapping.stitchScans(scans,XRF=False,XRD_cake=True,proposal=proposal,visit=visit,xrd_range=xrd_range,azi_range=azi_range)
            maps['cake_map'] = (maps['cake_map'].transpose(2,3,0,1)/ maps['I0_map']).transpose(2,3,0,1)
            
            process_folder = DM.getAzintFname(DM.findScan(groups[group][animal][0])).split('/azint/')[0]
            
            
            flat,chi = flatten(maps,nanangles,bkgtth,datatth,local_background)
            if use_phase_mask:
                mask = np.nansum(flat,axis=2)>phase_mask_limit
            else:
                mask_file =  f'{process_folder}/mask/{group}.h5'
                with h5py.File(mask_file, 'r') as mf:
                    mask = mf[f'mask/{animal}'][:]
            
            fitmap = DM.fitting.fitMesh(DM.fitting.circleGaussFit,chi,flat,mask,verbose=True)
            fit_params,names = DM.fitting.parseFitMesh(DM.fitting.parseCircleGaussFit,fitmap)
            save_folder =f'{process_folder}/aguf_fits/peak_{peak}'
            save_file = f'{save_folder}/{group}.h5'
            if not os.path.isdir(save_folder):
                os.makedirs(save_folder)
                os.chmod(save_folder,0o770)
            if not file_exists:
                file_exists=True
                if os.path.isfile(save_file):
                    os.system(f'rm {save_file}')
                with h5py.File(save_file, 'w') as sf:
                    sf.create_group(f'{animal}')
            with h5py.File(save_file, 'a') as sf:
                for i,name in enumerate(names):
                    sf.create_dataset(f'{animal}/{name}',data = fit_params[:,:,i])
            os.chmod(save_file,0o770)

## Create AGUF Figures

In [ ]:
for peak in peaks:
    print(f'Making figures for peak: {peak}')
    chi_shift = peaks[peak]['chi_shift']
    for group in groups:
        print(f'\tMaking figures for group: {group}')
        file_exists = False
        for animal in groups[group]:
            print(f'\t\tFigures for: {animal}')
            scans = groups[group][animal]
            process_folder = DM.getAzintFname(DM.findScan(groups[group][animal][0])).split('/azint/')[0]
            
            fit_folder =f'{process_folder}/aguf_fits/peak_{peak}'
            fit_file = f'{fit_folder}/{group}.h5'
            with h5py.File(fit_file, 'r') as ff:
                Hue = (ff[f'{animal}/H'][:]+chi_shift) % 1
                DoO = ff[f'{animal}/DoO'][:]
                I_tot = ff[f'{animal}/I_tot'][:]
    
            def _save_im(im,type):
                save_folder = f'{process_folder}/aguf_figures/peak_{peak}/{type}'
                save_file = f'{save_folder}/{group}_{animal}.png'
                if not os.path.isdir(save_folder):
                    os.makedirs(save_folder)
                    os.chmod(save_folder,0o770)
                plt.imsave(save_file,im)
                os.chmod(save_file,0o770)
            aguf_im = DM.mapping.combineMaps(Hue,DoO,I_tot/np.max(I_tot),True,[[10,15],[10,49]])
            _save_im(aguf_im,'full')
            aguf_im = DM.mapping.combineMaps(Hue,DoO!=0,DoO,True,[[10,15],[10,49]])
            _save_im(aguf_im,'DoO')
            aguf_im = DM.mapping.combineMaps(Hue,DoO!=0,I_tot/np.max(I_tot),True,[[10,15],[10,49]])
            _save_im(aguf_im,'I_tot')